In [32]:
#Importing packages
from selenium import webdriver
import pandas as pd
import nltk
from collections import Counter 
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold
import string
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
nltk.download('punkt')
import string
import operator
import io
import warnings
warnings.filterwarnings("ignore")

In [33]:
comments = pd.read_csv('test_set.csv', index_col=0)

In [34]:
#Create new column in DataFrame of word tokens from comments
comments['Tokens'] = comments['comments'].apply(nltk.tokenize.word_tokenize)
comments

,Date,user_id,comments,Tokens
0,"October 26, 2014 8:22AM",backy,My BMW is safer than hyundai,"[My, BMW, is, safer, than, hyundai]"
1,"October 26, 2014 9:42AM",brian125,"my bmw bmw is awesome, even better than Kia","[my, bmw, bmw, is, awesome, ,, even, better, t..."
2,"October 26, 2014 10:18AM",thebean,my lexus is safer than BMW,"[my, lexus, is, safer, than, BMW]"
3,"October 26, 2014 10:22AM",thebean,"my mercedes is great, but Lexus is better","[my, mercedes, is, great, ,, but, Lexus, is, b..."
4,"October 26, 2014 10:27AM",thebean,I drive a Lexus and it is way better,"[I, drive, a, Lexus, and, it, is, way, better]"
5,"October 26, 2014 10:30AM",thebean,"my audi, has smooth driving experience","[my, audi, ,, has, smooth, driving, experience]"
6,"October 26, 2014 10:34AM",thebean,"my wagonr, is safer","[my, wagonr, ,, is, safer]"
7,"October 26, 2014 10:37AM",brian125,"my honda, smooth driving experience","[my, honda, ,, smooth, driving, experience]"
8,"October 26, 2014 11:33AM",backy,"my mitsubishi, is safe","[my, mitsubishi, ,, is, safe]"


In [35]:
#Define Stop Words & Update to include some punctuation and car(s)
stop_words = nltk.corpus.stopwords.words('english')+ list(string.punctuation)

def removeSW(word_tokens):
    filtered_sentence = []
    for w in word_tokens:
        if w.lower() not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

#Apply the removeSW function to the word tokens in the 'comments' DF
comments['Tokens'] = comments['Tokens'].apply(removeSW)

In [36]:
models = pd.read_csv('car_models_and_brands.csv', header=None, names=['Brand','Model'])

#Create a list of all Models
models_list = models['Model'].tolist()

In [37]:
#Define a function to clean tokens & replace models with Brands
def token_cleaner(row):
    clean_tokens = []
    list_row = list(row) #Set --> List
    for word in list_row:
        word = word.lower()
        if word in models_list: #Change any models that appear to their corresponding Brands
            word = models['Brand'][models['Model']==word].tolist()[0]
            if word not in clean_tokens:
                clean_tokens.append(word.lower())
        else:
            if word not in clean_tokens:
                clean_tokens.append(word.lower())
    clean_tokens = [x.lower() for x in clean_tokens]
    return clean_tokens

comments['Tokens'] = comments['Tokens'].apply(token_cleaner)

In [38]:
#Define a function to clean tokens & replace models with Brands
def token_cleaner(row):
    clean_tokens = []
    list_row = list(row) #Set --> List
    for word in list_row:
        word = word.lower()
        if word in models_list: #Change any models that appear to their corresponding Brands
            word = models['Brand'][models['Model']==word].tolist()[0]
            if word not in clean_tokens:
                clean_tokens.append(word.lower())
        else:
            if word not in clean_tokens:
                clean_tokens.append(word.lower())
    clean_tokens = [x.lower() for x in clean_tokens]
    return clean_tokens

comments['Tokens'] = comments['Tokens'].apply(token_cleaner)

In [39]:
#Initialize empty dictionary
words_dict = dict()

#Iterate over each row of tokens and track frequency in the 'words_dict' dictionary
for row in comments['Tokens']:
    list_row = list(row) #Set --> List
    for word in row:
        word = word.lower() #Remove any capitalization
        if word in models_list: #Change any models that appear to their corresponding Brands
            word = models['Brand'][models['Model']==word].tolist()[0]

        if word in words_dict:
            words_dict[word] += 1
        else:
            words_dict[word] = 1

In [40]:
top_makes = dict()
for i in words_dict:
    if i in list(set(models['Brand'])):
        top_makes[i] = words_dict[i]

k = Counter(top_makes)
high = k.most_common(10)

top_brands = []
for i in high:
    top_brands.append(i[0])

df_freq = pd.DataFrame(index=top_brands, columns=['Frequency'])

for i in range(0,len(high)):
    df_freq['Frequency'][i] = high[i][1]

print('\n \033[1m' + 'Brands with Highest Frequency:' + '\033[0m')
display(df_freq)


 Brands with Highest Frequency:


,Frequency
bmw,3
hyundai,1
kia,1
mercedes benz,1
audi,1
honda,1
mitsubishi,1


In [41]:
#Create a list of the unique combinations of 2 brands that appeared in the top 10 most frequent list
top_brand_combos = list(itertools.combinations(top_brands,2))

In [42]:
#Define a function that will calculate Lift
def liftCalc(word_1,word_2):
    w1freq = sum(comments.apply(lambda x: word_1 in x['Tokens'], axis=1))
    w2freq = sum(comments.apply(lambda x: word_2 in x['Tokens'], axis=1))
    bothfreq = sum(comments.apply(lambda x: word_1 in x['Tokens'] and word_2 in x['Tokens'], axis=1))
    return float(bothfreq/len(comments))/((float(w1freq)/len(comments))*(float(w2freq)/len(comments)))

In [43]:
#Initialize empty dictionary for lift values
brand_lifts = dict()

#iterate over brand combinations, calculate lift, save to dictionary
for i in range(0,len(top_brand_combos)):
    a,b = top_brand_combos[i]
    brands = (a,b)
    lift = liftCalc(a,b)
    brand_lifts[brands] = lift

In [44]:
df_lifts = pd.DataFrame(columns=top_brands,index=top_brands)
for brand in top_brands:
    df_lifts[brand][brand] = '-'
for brands in brand_lifts:
    a,b = brands
    df_lifts[a][b] = (brand_lifts[brands])
    df_lifts[b][a] = '-'

In [45]:
#Lift values for brand associations
df_lifts

,bmw,hyundai,kia,mercedes benz,audi,honda,mitsubishi
bmw,-,-,-,-,-,-,-
hyundai,3.0,-,-,-,-,-,-
kia,3.0,0.0,-,-,-,-,-
mercedes benz,0.0,0.0,0.0,-,-,-,-
audi,0.0,0.0,0.0,0.0,-,-,-
honda,0.0,0.0,0.0,0.0,0.0,-,-
mitsubishi,0.0,0.0,0.0,0.0,0.0,0.0,-
